In [1]:
!pip install fhir.resources

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support


In [2]:
import base64
import csv
from datetime import date
from functools import reduce
from github import Github
import json
import pprint
import random
import uuid
from fhir.resources.bundle import Bundle
from fhir.resources.devicedefinition import DeviceDefinition
from IPython.core.debugger import set_trace

In [3]:
g = Github('access_token')
rdt_scan_repo = g.get_repo('cjpark87/rdt-scan')
img_path_prefix = 'app/src/main/res/drawable-nodpi/'

# generate reproducible UUIDs
rd = random.Random()
seed = "Wznza9%R$xs8eigsteuKg5pewMnAuFK8fWidZ2D9r8Fstb!Fpa%ovZs3o3yQQ%Jp4Lv2dS#eAbaH^*A@kxCTCSes*XzZfKXx6KRY9rgV!t^8#woP@GrpMHVnXJb3gQG*"

empty_flag = "ZZZ"

line_code_to_text = {
    "C": "Control",
    "T": "Test",
    "G": "IgG",
    "M": "IgM",
    "G+M": "IgG / IgM"
}

In [4]:

# generate reproducible UUIDs
# rd = random.Random()

# uuid.uuid4()

In [5]:
def is_valid(row):
    return(
        reduce(lambda x, y: x and row[y],
               ['Manufacturer', 'RDT_type', 'Lines'],
               True)
        and (row['Sampleqty drops'] or row['Sampleqty uL'])
    )        
        
def add_image(config):
    path = config['REF_IMG']
    if not path: return
    content = None
    try:
        content = rdt_scan_repo.get_contents('{}{}.jpg'.format(img_path_prefix, path)).decoded_content
    except:
        content = rdt_scan_repo.get_contents('{}{}.JPG'.format(img_path_prefix, path)).decoded_content
    config['REF_IMG'] = base64.b64encode(content).decode("utf-8") 
    return config

def build_device_definitions(upto=0, seed=seed):
    # seed reproducible UUIDs per call
    rd.seed(seed)

    device_definitions = []

    with open('fhir-deviceDefinitions-list.csv') as f:
        csv_reader = csv.DictReader(f)
        for i, row in enumerate(csv_reader):
            if upto and i > upto:
                break
            if not is_valid(row):
                print("skipping row {}".format(i))
                continue

            dd_json = {}

            # generate a UUID 4 string with the seed RNG
            dd_json["identifier"] = [
                {
                    "value": str(uuid.UUID(int=rd.getrandbits(128)))
                }
            ]
            dd_json["manufacturerString"] = row['Manufacturer']
            dd_json["deviceName"] = [{
                "name": row['RDT_type'],
                "type": "user-friendly-name"
            }]
            dd_json["udiDeviceIdentifier"] = []
            dd_json["capability"] = []
            dd_json["property"] = []

            if row['GTIN']:
                dd_json["udiDeviceIdentifier"].append({
                    "deviceIdentifier": row['GTIN'],
                    "issuer": "urn:gtin",
                    "jurisdiction": "urn:global"
                })

            if row['Product Code']:
                dd_json["udiDeviceIdentifier"].append({
                    "deviceIdentifier": row['Product Code'],
                    "issuer": "urn:manufacturer",
                    "jurisdiction": "urn:global"
                })

            # define capabilities
            if row['Steps']:
                dd_json["capability"].append({
                    "type": {
                        "text":"instructions"
                    },
                    "description": [
                        {
                            "text": step.strip()
                        } for step in row['Steps'].split('/')
                    ]
                })

            if row['Lines']:
                try:
                    dd_json["capability"].append({
                        "type": {
                            "text":"lines"
                        },
                        "description": [
                            {
                                "coding": [{"code":line}],
                                "text": line_code_to_text[line]
                            } for line in row['Lines'].split('/')
                        ]
                    })
                except KeyError:
                    print("Line code to text key '{}' not found, skipping row {}".format(line, i))
                    continue

            if row['Sampletypes']:
                dd_json["capability"].append({
                    "type": {
                        "text":"sample types"
                    },
                    "description": [
                        {
                            "text": sample_type.strip()
                        } for sample_type in row['Sampletypes'].split('/')
                    ]
                })

            # define timing property
            timing_value_quantities = []
            
            if row['minimumtimeforresult']:
                timing_value_quantities.append({
                    "value": float(row['minimumtimeforresult']),
                    "comparator": ">=",
                    "unit": "minutes"
                })

            if row['maximumtimeforresult']:
                timing_value_quantities.append({
                    "value": float(row['maximumtimeforresult']),
                    "comparator": "<=",
                    "unit": "minutes"
                })
                
            if len(timing_value_quantities):
                dd_json["property"].append({
                    "type": {
                        "text": "Time For Result"
                    },
                    "valueQuantity": timing_value_quantities
                })

            # define temperature property
            temperature_value_quantities = []
            
            if row['low_temp']:
                temperature_value_quantities.append({
                    "value": float(row['low_temp']),
                    "comparator": ">=",
                    "unit": "degrees celsius"
                })

            if row['high_temp']:
                temperature_value_quantities.append({
                    "value": float(row['high_temp']),
                    "comparator": "<=",
                    "unit": "degrees celsius"
                })
                
            if len(temperature_value_quantities):
                dd_json["property"].append({
                    "type": {
                        "text": "Temperature"
                    },
                    "valueQuantity": temperature_value_quantities
                })

            # define sample quantity property
            sample_quantity_value_quantities = []

            if row['Sampleqty drops']:
                sample_quantity_value_quantities.append({
                    "value": float(row['Sampleqty drops']),
                    "unit": "drop"
                })

            if row['Sampleqty uL']:
                sample_quantity_value_quantities.append({
                    "value": float(row['Sampleqty uL']),
                    "unit": "microliter"
                })


            if len(sample_quantity_value_quantities):
                dd_json["property"].append({
                    "type": {
                        "text": "Sample Quantity"
                    },
                    "valueQuantity": sample_quantity_value_quantities
                })

            # define buffer quantity property
            buffer_quantity_value_quantities = []

            if row['Bufferqty drops']:
                buffer_quantity_value_quantities.append({
                    "value": float(row['Bufferqty drops']),
                    "comparator": ">=",
                    "unit": "drop"
                })

            if row['Bufferqty drops max']:
                buffer_quantity_value_quantities.append({
                    "value": float(row['Bufferqty drops max']),
                    "comparator": "<=",
                    "unit": "drop"
                })

            if row['Bufferqty uL']:
                buffer_quantity_value_quantities.append({
                    "value": float(row['Bufferqty uL']),
                    "unit": "microliter"
                })

            if len(buffer_quantity_value_quantities):
                dd_json["property"].append({
                    "type": {
                        "text": "Buffer Quantity"
                    },
                    "valueQuantity": buffer_quantity_value_quantities
                })

            # define rdt scan configuration property
            if row['RDTScan Configuration']:
                rdtscan_json = json.loads(row['RDTScan Configuration'])
                rdtscan_json = add_image(rdtscan_json)
                value_code = [{
                    "coding": [
                        {
                            "code": key
                        }
                    ],
                    "text": str(value)
                } for key, value in rdtscan_json.items()]
                    
                dd_json["property"].append({
                    "type": {
                        "text": "RDTScan Configuration"
                    },
                    "valueCode": value_code
                })
            
            # convert JSON to FHIR class
            dd = DeviceDefinition(dd_json)
            device_definitions.append(dd)
            
    return device_definitions

device_definitions = build_device_definitions()

device_definitions_json = [{"resource": x.as_json()} for x in device_definitions]

bundle = Bundle({
    "identifier": {
        "value": "rdt-og-device-definitions"
      },
    "type": "collection",
    "entry": device_definitions_json
})

with open('device_definitions-rdt-og-{}.json'.format(date.today().isoformat()), 'w') as f:
    f.write(json.dumps(bundle.as_json(), indent=4))

pprint.pprint(device_definitions_json)

skipping row 15
skipping row 16
skipping row 17
skipping row 18
skipping row 19
skipping row 20
skipping row 21
skipping row 22
skipping row 23
skipping row 24
skipping row 25
skipping row 26
skipping row 27
skipping row 28
skipping row 29
skipping row 30
skipping row 31
skipping row 32
skipping row 33
skipping row 34
skipping row 35
skipping row 36
skipping row 37
skipping row 38
skipping row 39
skipping row 40
skipping row 41
skipping row 42
skipping row 43
skipping row 44
skipping row 45
skipping row 46
skipping row 47
skipping row 48
skipping row 49
skipping row 50
skipping row 51
skipping row 52
skipping row 53
skipping row 54
skipping row 55
skipping row 56
skipping row 57
skipping row 58
skipping row 59
skipping row 60
[{'resource': {'capability': [{'description': [{'text': 'Collect blood sample'}],
                               'type': {'text': 'instructions'}},
                              {'description': [{'coding': [{'code': 'C'}],
                                         

                                       {'deviceIdentifier': 'W195',
                                        'issuer': 'urn:manufacturer',
                                        'jurisdiction': 'urn:global'}]}},
 {'resource': {'capability': [{'description': [{'text': 'Collect blood sample'}],
                               'type': {'text': 'instructions'}},
                              {'description': [{'coding': [{'code': 'C'}],
                                                'text': 'Control'},
                                               {'coding': [{'code': 'G'}],
                                                'text': 'IgG'},
                                               {'coding': [{'code': 'M'}],
                                                'text': 'IgM'}],
                               'type': {'text': 'lines'}},
                              {'description': [{'text': 'Whole Blood'}],
                               'type': {'text': 'sample types'}}],
               'dev

                             'valueCode': [{'coding': [{'code': u'REF_IMG'}],
                                            'text': '...'},
                                           {'coding': [{'code': u'TOP_LINE_POSITION'}],
                                            'text': '[285, 120]'},
                                           {'coding': [{'code': u'TOP_LINE_NAME'}],
                                            'text': 'Control'},
                                           {'coding': [{'code': u'VIEW_FINDER_SCALE'}],
                                            'text': '0.6'},
                                           {'coding': [{'code': u'LINE_INTENSITY'}],
                                            'text': '85'},
                                           {'coding': [{'code': u'RESULT_WINDOW_BOTTOM_RIGHT'}],
                                            'text': '[390, 105]'},
                                           {'coding': [{'code': u'RESULT_WINDOW_TOP_LEFT'}],
          

                            {'type': {'text': 'RDTScan Configuration'},
                             'valueCode': [{'coding': [{'code': u'REF_IMG'}],
                                            'text': '...'},
                                           {'coding': [{'code': u'TOP_LINE_POSITION'}],
                                            'text': '[280, 125]'},
                                           {'coding': [{'code': u'TOP_LINE_NAME'}],
                                            'text': 'Control'},
                                           {'coding': [{'code': u'VIEW_FINDER_SCALE'}],
                                            'text': '0.6'},
                                           {'coding': [{'code': u'LINE_INTENSITY'}],
                                            'text': '85'},
                                           {'coding': [{'code': u'RESULT_WINDOW_BOTTOM_RIGHT'}],
                                            'text': '[395, 135]'},
                               

                                               {'comparator': '<=',
                                                'unit': 'drop',
                                                'value': 2.0}]},
                            {'type': {'text': 'RDTScan Configuration'},
                             'valueCode': [{'coding': [{'code': u'REF_IMG'}],
                                            'text': '...'},
                                           {'coding': [{'code': u'TOP_LINE_POSITION'}],
                                            'text': '[280, 125]'},
                                           {'coding': [{'code': u'TOP_LINE_NAME'}],
                                            'text': 'Control'},
                                           {'coding': [{'code': u'VIEW_FINDER_SCALE'}],
                                            'text': '0.6'},
                                           {'coding': [{'code': u'LINE_INTENSITY'}],
                                            'text': '85'}

...

                            {'type': {'text': 'RDTScan Configuration'},
                             'valueCode': [{'coding': [{'code': u'REF_IMG'}],
                                            'text': '...'},
                                           {'coding': [{'code': u'TOP_LINE_POSITION'}],
                                            'text': '[275, 105]'},
                                           {'coding': [{'code': u'TOP_LINE_NAME'}],
                                            'text': 'Control'},
                                           {'coding': [{'code': u'VIEW_FINDER_SCALE'}],
                                            'text': '0.6'},
                                           {'coding': [{'code': u'LINE_INTENSITY'}],
                                            'text': '85'},
                                           {'coding': [{'code': u'RESULT_WINDOW_BOTTOM_RIGHT'}],
                                            'text': '[370, 115]'},
                               

               'property': [{'type': {'text': 'Time For Result'},
                             'valueQuantity': [{'comparator': '>=',
                                                'unit': 'minutes',
                                                'value': 10.0},
                                               {'comparator': '<=',
                                                'unit': 'minutes',
                                                'value': 15.0}]},
                            {'type': {'text': 'Temperature'},
                             'valueQuantity': [{'comparator': '>=',
                                                'unit': 'degrees celsius',
                                                'value': 2.0},
                                               {'comparator': '<=',
                                                'unit': 'degrees celsius',
                                                'value': 30.0}]},
                            {'type': {'text': 'Sample Qua

 {'resource': {'capability': [{'description': [{'text': 'Collect blood sample'}],
                               'type': {'text': 'instructions'}},
                              {'description': [{'coding': [{'code': 'C'}],
                                                'text': 'Control'},
                                               {'coding': [{'code': 'T'}],
                                                'text': 'Test'}],
                               'type': {'text': 'lines'}},
                              {'description': [{'text': 'Whole Blood'}],
                               'type': {'text': 'sample types'}}],
               'deviceName': [{'name': 'SD Biosensor STANDARD Q Syphilis Ab',
                               'type': 'user-friendly-name'}],
               'identifier': [{'value': '9d92f9b2-22d0-7509-dc55-04fcf3af7f81'}],
               'manufacturerString': 'SD Biosensor',
               'property': [{'type': {'text': 'Time For Result'},
                           

               'property': [{'type': {'text': 'Time For Result'},
                             'valueQuantity': [{'comparator': '>=',
                                                'unit': 'minutes',
                                                'value': 5.0},
                                               {'comparator': '<=',
                                                'unit': 'minutes',
                                                'value': 20.0}]},
                            {'type': {'text': 'Temperature'},
                             'valueQuantity': [{'comparator': '>=',
                                                'unit': 'degrees celsius',
                                                'value': 2.0},
                                               {'comparator': '<=',
                                                'unit': 'degrees celsius',
                                                'value': 40.0}]},
                            {'type': {'text': 'Sample Quan

 {'resource': {'capability': [{'description': [{'text': 'Gently rotate and push the swab into the nasal cavity until the nasal turbinate is blocked (about 2.0-2.5cm from the nostril), then press the swab against the nasal wall for three times and remove the swab. The sample should be treated with the virus sampling solution or the sample extraction solution provided with this kit as soon as possible after collection. And complete the test in 5 minutes.'},
                                               {'text': 'Put the swab into the sampling tube and rotate it about 10 times to make the sample dissolve in the solution as much as possible.'},
                                               {'text': 'Open the aluminum foil bag along the tear mouth and take the test cassette out, and put on a flat surface.'}],
                               'type': {'text': 'instructions'}},
                              {'description': [{'coding': [{'code': 'C'}],
                                         